In [1]:
# packages

# sql
import sqlite3

import pandas as pd

import re
import os

 # Create the database : need to change the CSV and future database paths

In [94]:
# Connect to SQLite database (replace 'your_database.db' with your database name)
conn = sqlite3.connect('DVM_car.db')

#----------------------------------------------CHANGE THE PATH------------------------------------------------------
# path to the folder containing unpacked csv's
csv_folder = r'C:\Users\Olivier\Desktop\data'
#-------------------------------------------------------------------------------------------------------------------

listed = []

# Get a list of CSV files in the folder
csv_files = [file for file in os.listdir(csv_folder) if file.endswith('.csv')]

# Loop through each CSV file
for csv_file in csv_files:
    
    # Read the CSV into a DataFrame
    csv_path = os.path.join(csv_folder, csv_file)
    df = pd.read_csv(csv_path, low_memory=False)
    
    # Extract table name from the CSV file name
    table_name = os.path.splitext(csv_file)[0]
    df.replace({'\$\$': '**'}, regex=True, inplace=True)
    # Create a table in the database based on DataFrame columns
    df.to_sql(table_name, conn, index=False, if_exists='replace')

# Commit the changes and close the connection
conn.commit()
#conn.close()

In [67]:
df
datatable = df.replace({'\$\$': '**'}, regex=True)

,Genmodel_ID,Image_ID,Image_name,Predicted_viewpoint,Quality_check
0,2_1,2_1**1**1,Abarth**124 Spider**2017**Blue**2_1**1**image_...,45,NaN
1,2_1,2_1**10**11,Abarth**124 Spider**2017**Blue**2_1**10**image...,45,NaN
2,2_1,2_1**10**16,Abarth**124 Spider**2017**Blue**2_1**10**image...,0,N
3,2_1,2_1**10**18,Abarth**124 Spider**2017**Blue**2_1**10**image...,270,NaN
4,2_1,2_1**10**2,Abarth**124 Spider**2017**Blue**2_1**10**image...,180,NaN
...,...,...,...,...,...
1451779,99_1,99_1**3**7,Zenos**E10**2016**Grey**99_1**3**image_7.jpg,90,NaN
1451780,99_1,99_1**1**0,Zenos**E10**2016**Red**99_1**1**image_0.jpg,225,NaN
1451781,99_1,99_1**1**3,Zenos**E10**2016**Red**99_1**1**image_3.jpg,315,NaN
1451782,99_1,99_1**1**4,Zenos**E10**2016**Red**99_1**1**image_4.jpg,45,NaN


# Merge datatable to create a single table

In [77]:
conn = sqlite3.connect('DVM_car.db')

query = "SELECT * FROM autosWithNames"
df_autosWithNames = pd.read_sql_query(query, conn)

query2 = "SELECT * FROM `Ad_table (extra)`"
df_Ad_table = pd.read_sql_query(query2, conn)

query3 = "SELECT * FROM Image_table"
df_Images = pd.read_sql_query(query3, conn)

In [3]:
df_autosWithNames

,brand,model_name,model_id,model_year,bodytype
0,Land Rover,Range Rover,47_13,2016,SUV
1,Renault,Trafic,75_17,2015,Combi Van
2,Vauxhall,Astra,94_5,2003,Hatchback
3,Volkswagen,Touareg,95_28,2013,SUV
4,Audi,A1,7_1,2017,Hatchback
...,...,...,...,...,...
3334,Lexus,RX 450h,48_24,2012,SUV
3335,MINI,Unspecified,54_11,2003,Saloon
3336,Toyota,Land Cruiser,92_21,2017,SUV
3337,Ford,Grand Tourneo Connect,29_21,2014,MPV


In [100]:
df_Ad_table.columns

Index(['Maker', ' Genmodel', ' Genmodel_ID', 'Adv_ID', 'Adv_year', 'Adv_month',
       'Color', 'Reg_year', 'Bodytype', 'Runned_Miles', 'Engin_size',
       'Gearbox', 'Fuel_type', 'Price', 'Engine_power', 'Annual_Tax',
       'Wheelbase', 'Height', 'Width', 'Length', 'Average_mpg', 'Top_speed',
       'Seat_num', 'Door_num'],
      dtype='object')

In [111]:
df_Ad_table['image_link'] = df_Ad_table['Maker']  +'**'+df_Ad_table[' Genmodel'] +'**'+df_Ad_table['Adv_year'].astype(str)+'**'+df_Ad_table['Color']+'**' +df_Ad_table['Adv_ID']  

In [112]:
df_Ad_table

,Maker,Genmodel,Genmodel_ID,Adv_ID,Adv_year,Adv_month,Color,Reg_year,Bodytype,Runned_Miles,...,Annual_Tax,Wheelbase,Height,Width,Length,Average_mpg,Top_speed,Seat_num,Door_num,image_link
0,Bentley,Arnage,10_1,10_1$$1,2018,4,Silver,2000.0,Saloon,60000,...,None,3116.0,1515.0,2125.0,5390.0,None,None,5.0,4.0,Bentley**Arnage**2018**Silver**10_1$$1
1,Bentley,Arnage,10_1,10_1$$2,2018,6,Grey,2002.0,Saloon,44000,...,315,3116.0,1515.0,2125.0,5390.0,13.7 mpg,179 mph,5.0,4.0,Bentley**Arnage**2018**Grey**10_1$$2
2,Bentley,Arnage,10_1,10_1$$3,2017,11,Blue,2002.0,Saloon,55000,...,315,3116.0,1515.0,2125.0,5390.0,14.7 mpg,155 mph,5.0,4.0,Bentley**Arnage**2017**Blue**10_1$$3
3,Bentley,Arnage,10_1,10_1$$4,2018,4,Green,2003.0,Saloon,14000,...,None,3116.0,1515.0,2125.0,5390.0,None,None,5.0,4.0,Bentley**Arnage**2018**Green**10_1$$4
4,Bentley,Arnage,10_1,10_1$$5,2017,11,Grey,2003.0,Saloon,61652,...,None,3116.0,1515.0,2125.0,5390.0,None,None,5.0,4.0,Bentley**Arnage**2017**Grey**10_1$$5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268250,Westfield,Sport,97_1,97_1$$1,2018,5,Yellow,2006.0,Convertible,1800,...,None,NaN,NaN,NaN,NaN,None,None,2.0,NaN,Westfield**Sport**2018**Yellow**97_1$$1
268251,Westfield,Sport,97_1,97_1$$2,2018,5,Yellow,2006.0,Convertible,2009,...,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,Westfield**Sport**2018**Yellow**97_1$$2
268252,Zenos,E10,99_1,99_1$$1,2018,3,Red,2016.0,Convertible,6,...,None,NaN,NaN,NaN,NaN,None,None,2.0,NaN,Zenos**E10**2018**Red**99_1$$1
268253,Zenos,E10,99_1,99_1$$2,2018,3,Green,2016.0,Convertible,1538,...,None,NaN,NaN,NaN,NaN,None,None,2.0,NaN,Zenos**E10**2018**Green**99_1$$2


In [95]:
query3 = "SELECT * FROM Image_table"
df_Images = pd.read_sql_query(query3, conn)

In [96]:
df_Images

,Genmodel_ID,Image_ID,Image_name,Predicted_viewpoint,Quality_check
0,2_1,2_1**1**1,Abarth**124 Spider**2017**Blue**2_1**1**image_...,45,None
1,2_1,2_1**10**11,Abarth**124 Spider**2017**Blue**2_1**10**image...,45,None
2,2_1,2_1**10**16,Abarth**124 Spider**2017**Blue**2_1**10**image...,0,N
3,2_1,2_1**10**18,Abarth**124 Spider**2017**Blue**2_1**10**image...,270,None
4,2_1,2_1**10**2,Abarth**124 Spider**2017**Blue**2_1**10**image...,180,None
...,...,...,...,...,...
1451779,99_1,99_1**3**7,Zenos**E10**2016**Grey**99_1**3**image_7.jpg,90,None
1451780,99_1,99_1**1**0,Zenos**E10**2016**Red**99_1**1**image_0.jpg,225,None
1451781,99_1,99_1**1**3,Zenos**E10**2016**Red**99_1**1**image_3.jpg,315,None
1451782,99_1,99_1**1**4,Zenos**E10**2016**Red**99_1**1**image_4.jpg,45,None


The path to the good folder is 'Confirmed_fronts/Confirmed_fronts/Brand/year'
Once we got to here, we can load the image, which is called as Brand//Genmodel//Year//Color//Adv_ID//image_id

Adv_ID = Genmodel_ID + a number

Both last elements aren't in the Images table, gonna have to find a solution to process all cars that match all other elements

In [7]:
# close the connection to database
conn.close()